In [1]:
import anndata
import numpy as np
import scipy as sp
import scvelo as scv
import scanpy as sc
import sys
import pandas as pd
import torch
import os.path
sys.path.append('../')
import topovelo as tpv
import matplotlib.pyplot as plt
from scipy.io import mmread, mmwrite
from numpy import savetxt
%load_ext autoreload
%autoreload 2

In [2]:
dataset = 'Dentategyrus'
root = "/scratch/blaauw_root/blaauw1/gyichen"
adata = anndata.read_h5ad(f'{root}/data/{dataset}_pp.h5ad')

In [8]:
adata.var.head()

,gene_count_corr,means,dispersions,dispersions_norm,highly_variable
index,,,,,
Tcea1,-0.0993,0.105639,0.374601,0.332928,True
Rb1cc1,-0.0002,0.215175,1.007291,2.114861,True
Prex2,-0.1030,0.050708,0.445865,0.551174,True
Tmeff2,-0.0320,0.437797,0.998461,1.449496,True
Gm10561,-0.1113,0.025952,1.188372,2.825089,True


In [3]:
model_path_base = f'{root}/checkpoints/{dataset}'
figure_path_base = f'{root}/figures/{dataset}'
data_path = f'{root}/data/velovae/continuous/{dataset}'
gene_plot = tpv.sample_genes(adata, 4, 'means')

In [5]:
figure_path = f'{figure_path_base}/Graph'
model_path = f'{model_path_base}/Graph'

torch.manual_seed(2022)
np.random.seed(2022)

vae = tpv.VAE(adata, 
              tmax=20, 
              dim_z=5, 
              device='cuda:0')

Estimating ODE parameters...


  0%|          | 0/1000 [00:00<?, ?it/s]

Detected 255 velocity genes.
Estimating the variance...


  0%|          | 0/1000 [00:00<?, ?it/s]

Initialization using the steady-state and dynamical models.
Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/1000 [00:00<?, ?it/s]

3 clusters detected based on gene co-expression.
(0.70, 0.8276461110034795), (0.30, 0.33719022151889355)
(0.50, 0.7812572084899341), (0.50, 0.2642154925943826)
(0.55, 0.7841723167950118), (0.45, 0.2521193684810272)
KS-test result: [0. 0. 0.]
Initial induction: 609, repression: 391/1000
Gaussian Prior.


In [6]:
graph = adata.uns['neighbors']['connectivities']

In [ ]:
!nvidia-smi

In [ ]:
vae.train(adata, graph, plot=True, gene_plot=gene_plot, figure_path=figure_path)
#vae.save_model(model_path, 'encoder', 'decoder')
#vae.save_anndata(adata, 'velovae', data_path, file_name=f'{dataset}.h5ad')

--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 17,test iteration: 32
Epoch 1: Train ELBO = -38464.203,	Test ELBO = -40606.293,	Total Time =   0 h :  0 m : 21 s


In [32]:
vae.graph_data.neighbor_indices.dtype

dtype('int64')

In [34]:
vae.encoder.bias_1.dtype

torch.float32

In [46]:
vae.encoder.bias_2.dtype

torch.float64